In [ ]:
import sfacts as sf

In [ ]:
%%bash
sfacts filter_mgen \
    --min-minor-allele-freq 0.05 \
    --min-horizontal-cvrg 0.1 \
    --random-seed 0 \
    sim.mgen.nc sim.filt.mgen.nc
sfacts fit \
    --verbose \
    --model-structure ssdd3_with_error \
    --num-strains 15 --num-positions 500 \
    --random-seed 0 \
    sim.filt.mgen.nc sim.filt.fit.world.nc

In [ ]:
fit = sf.data.World.load('sim.filt.fit.world.nc')
sf.plot.plot_community(fit, row_linkage_func=lambda w: w.genotypes.linkage("strain"), col_linkage_func=lambda w: w.metagenotypes.linkage("sample"))
sf.plot.plot_genotype(fit, row_linkage_func=lambda w: w.genotypes.linkage("strain"))
sf.plot.plot_metagenotype(fit, col_linkage_func=lambda w: w.metagenotypes.linkage("sample"))

In [ ]:
%bash
sfacts fit_genotype \
    --verbose \
    --model-structure ssdd3_with_error \
    --num-positionsB 1000 \
    --hyperparameters gamma_hyper=1e-5 \
    --block-number 0 \
    --random-seed=0 \
    sim.filt.fit.world.nc sim.filt.mgen.nc sim.filt.fit.refit.world.nc

In [ ]:
%%bash
sfacts concatenate_genotype_chunks \
            --metagenotype sim.filt.mgen.nc \
            --community sim.filt.fit.world.nc \
            --outpath sim.filt.fit.refit.world.nc \
            sim.filt.fit.refit.geno.nc

In [ ]:
%%bash
sfacts dump --tsv sim.filt.fit.refit.world.nc \
    --genotype sim.filt.fit.refit.geno.tsv \
    --community sim.filt.fit.refit.comm.tsv

head sim.filt.fit.refit.geno.tsv sim.filt.fit.refit.comm.tsv